In [5]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import folium.features
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [9]:
def generate_heat_map(path):
    df = pd.read_csv(path)
    # Filter for donations data
    data2 = df.dropna(subset=['City', 'Donation Amount'])

    # Aggregate donations by city
    aggregated_donations2 = data2.groupby('City')['Donation Amount'].sum().reset_index()

    # Initialize geolocator
    geolocator = Nominatim(user_agent="donation_heatmap")

    # Geocode function to get latitude and longitude for each city
    def geocode_city(city, retries=3):
        for attempt in range(retries):
            try:
                location = geolocator.geocode(f"{city}, USA", timeout=10)
                if location:
                    return location.latitude, location.longitude
                else:
                    return None, None
            except GeocoderTimedOut:
                if attempt < retries - 1:
                    time.sleep(2)  # Wait before retrying
                    continue
            return None, None

    # Add latitude and longitude columns to the aggregated donations DataFrame
    aggregated_donations2[['Latitude', 'Longitude']] = aggregated_donations2['City'].apply(
        lambda city: pd.Series(geocode_city(city))
    )

    # Drop rows where geocoding failed
    aggregated_donations2 = aggregated_donations2.dropna(subset=['Latitude', 'Longitude'])

    # Create the base map
    base_map = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

    # Prepare data for the heat map (latitude, longitude, donation amount)
    heat_data2 = aggregated_donations2[['Latitude', 'Longitude', 'Donation Amount']].values.tolist()

    # Add the heat map to the base map
    HeatMap(heat_data2, radius=15).add_to(base_map)

    # Create a custom feature group for city information
    city_info = folium.FeatureGroup(name='City Donations')

    # Add custom hover functionality
    for index, row in aggregated_donations2.iterrows():
        # Create a custom JavaScript tooltip
        tooltip = folium.Popup(
            f"City: {row['City']}<br>Total Donations: ${row['Donation Amount']:,.2f}",
            max_width=300
        )
        
        # Add a transparent marker to attach the tooltip
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=0,  # Completely invisible
            popup=tooltip,
            color='transparent',
            fill=False
        ).add_to(city_info)

    # Add the feature group to the map
    city_info.add_to(base_map)

    # Save and display the map
    #base_map.save("heat_map -" + path)
    return base_map

In [ ]:
# Annual Campaigns
heatmap_2018_annual = generate_heat_map("annual campaign/2018 Annual Campaign - Archived.csv")
heatmap_2019_annual = generate_heat_map("annual campaign/2019 Annual Campaign - Archived.csv")
heatmap_2020_annual = generate_heat_map("annual campaign/2020 Annual Campaign - Archived.csv")
heatmap_2021_annual = generate_heat_map("annual campaign/2021 Annual Campaign - Archived.csv")
heatmap_2022_annual = generate_heat_map("annual campaign/2022 Annual Campaign - Active.csv")
heatmap_2023_annual = generate_heat_map("annual campaign/2023 Annual Campaign - Active.csv")
heatmap_2024_annual = generate_heat_map("annual campaign/2024 Annual Campaign - Active.csv")

# Bottoms Up Ball
heatmap_2022_bottoms_up_ball = generate_heat_map("Bottoms Up Ball/2022 Bottoms Up Ball - Active.csv")
heatmap_2023_bottoms_up_ball = generate_heat_map("Bottoms Up Ball/2023 Bottoms Up Ball - Active.csv")

# Diaper Drives
heatmap_2021_diaper_drive = generate_heat_map("Diaper Drive/2021 NBC 15 Community Diaper Driver.csv")
heatmap_2022_diaper_drive = generate_heat_map("Diaper Drive/2022 NBC 15 Community Diaper Driver.csv")
heatmap_2023_diaper_drive = generate_heat_map("Diaper Drive/2023 NBC 15 Community Diaper Driver.csv")
heatmap_2024_diaper_drive = generate_heat_map("Diaper Drive/2024 WMTV Diaper Drive.csv")